##  Notebook version of the translate.py program from Tensorflow
It does not do the training but it is designed to allow you to interactivly poke and prod the model.  To use this you need to run the tranlate.py program with

python translate.py
  --data_dir [your_data_directory] --train_dir [checkpoints_directory]
  --size=256 --num_layers=2 --steps_per_checkpoint=50
  
then set the paths to your_data_directory and the checkpoints_directory in mydata_dir and my_traindir variables below.

this program is effectively the same as running translate.py in decode mode.

python translate.py --decode
  --data_dir [your_data_directory] --train_dir [checkpoints_directory]


See the copyright for translate.py  

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os
import random
import sys
import time

import tensorflow.python.platform

import numpy as np
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
#for these make sure PYTHONPATH is set.
import data_utils
import seq2seq_model

In [2]:
!echo $PYTHONPATH

/home/dbgannon/tensorflow-master/tensorflow/models/rnn/translate


In [3]:
mylearning_rate = 0.5
mylearning_rate_decay_factor = 0.99
mymax_gradient_norm = 5.0
mybatch_size= 64
mysize = 256
mynum_layers = 2
myen_vocab_size = 40000
myfr_vocab_size = 40000
mydata_dir = "/datadrive/data"
mytrain_dir = "/datadrive/tmp"
mymax_train_data_size = 0
mysteps_per_checkpoint = 50
mydecode = True
myself_test = False
_buckets = [(5, 10), (10, 15), (20, 25), (40, 50)]

In [4]:
def read_data(source_path, target_path, max_size=None):
  """Read data from source and target files and put into buckets.
  Args:
    source_path: path to the files with token-ids for the source language.
    target_path: path to the file with token-ids for the target language;
      it must be aligned with the source file: n-th line contains the desired
      output for n-th line from the source_path.
    max_size: maximum number of lines to read, all other will be ignored;
      if 0 or None, data files will be read completely (no limit).
  Returns:
    data_set: a list of length len(_buckets); data_set[n] contains a list of
      (source, target) pairs read from the provided data files that fit
      into the n-th bucket, i.e., such that len(source) < _buckets[n][0] and
      len(target) < _buckets[n][1]; source and target are lists of token-ids.
  """
  data_set = [[] for _ in _buckets]
  with tf.gfile.GFile(source_path, mode="r") as source_file:
    with tf.gfile.GFile(target_path, mode="r") as target_file:
      source, target = source_file.readline(), target_file.readline()
      counter = 0
      while source and target and (not max_size or counter < max_size):
        counter += 1
        if counter % 100000 == 0:
          print("  reading data line %d" % counter)
          sys.stdout.flush()
        source_ids = [int(x) for x in source.split()]
        target_ids = [int(x) for x in target.split()]
        target_ids.append(data_utils.EOS_ID)
        for bucket_id, (source_size, target_size) in enumerate(_buckets):
          if len(source_ids) < source_size and len(target_ids) < target_size:
            data_set[bucket_id].append([source_ids, target_ids])
            break
        source, target = source_file.readline(), target_file.readline()
  return data_set


In [5]:
def create_model(session, forward_only):
  """Create translation model and initialize or load parameters in session."""
  model = seq2seq_model.Seq2SeqModel(
      myen_vocab_size, myfr_vocab_size, _buckets,
      mysize, mynum_layers, mymax_gradient_norm, mybatch_size,
      mylearning_rate, mylearning_rate_decay_factor,
      forward_only=forward_only)
  ckpt = tf.train.get_checkpoint_state(mytrain_dir)
  model.saver.restore(session, ckpt.model_checkpoint_path)
  return model


In [6]:
sess = tf.Session()
# Create model and load parameters.
model = create_model(sess, True)
model.batch_size = 1  # We decode one sentence at a time.



In [7]:
# Load vocabularies.
en_vocab_path = os.path.join(mydata_dir,
                             "vocab%d.en" % myen_vocab_size)
fr_vocab_path = os.path.join(mydata_dir,
                             "vocab%d.fr" % myfr_vocab_size)
en_vocab, rev_en_vocab = data_utils.initialize_vocabulary(en_vocab_path)
_, rev_fr_vocab = data_utils.initialize_vocabulary(fr_vocab_path)


In [8]:
sentence = " I am a person. "


In [9]:
token_ids = data_utils.sentence_to_token_ids(sentence, en_vocab)
      # Which bucket does it belong to?
bucket_id = min([b for b in xrange(len(_buckets))if _buckets[b][0] > len(token_ids)])
      # Get a 1-element batch to feed the sentence to the model.
encoder_inputs, decoder_inputs, target_weights = model.get_batch(
          {bucket_id: [(token_ids, [])]}, bucket_id)


In [10]:
_, _, output_logits = model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id, True)

In [11]:
len(output_logits)

15

In [12]:
outputs = [int(np.argmax(logit, axis=1)) for logit in output_logits]

In [13]:
if data_utils.EOS_ID in outputs:
    outputs = outputs[:outputs.index(data_utils.EOS_ID)]


In [14]:
print(" ".join([rev_fr_vocab[output] for output in outputs]))

Je suis une personne .
